# Time series forecasting with Tensorflow and Aeon

In [ ]:
import tensorflow as tf
from scikeras.wrappers import KerasRegressor

from matplotlib import pyplot as plt
from aeon.forecasting.compose import make_reduction
from aeon.datasets import load_airline
from aeon.utils.plotting import plot_series
import numpy as np
from aeon.forecasting.model_selection import temporal_train_test_split

In [ ]:
y = load_airline()
y_train, y_test = temporal_train_test_split(y, test_size=36)
# we will try to forecast y_test from y_train
fh = np.arange(len(y_test)) + 1

In [ ]:
def get_clf_model(hidden_layer_sizes, meta, compile_kwargs):

    model = tf.keras.Sequential()
    input_layer = tf.keras.layers.Input(shape=(meta["n_features_in_"]))
    model.add(input_layer)

    for i, hidden_layer_size in enumerate(hidden_layer_sizes):
        hidden_layer = tf.keras.layers.Dense(
            units=hidden_layer_size, activation="relu", name=f"hidden{i+1}"
        )
        model.add(hidden_layer)

    output_layer = tf.keras.layers.Dense(units=1, name="output")
    model.add(output_layer)

    model.compile(
        optimizer=compile_kwargs["optimizer"],
        loss="mean_squared_error",
        metrics=[tf.keras.metrics.mean_absolute_percentage_error],
    )

    return model

In [ ]:
tf.keras.backend.clear_session()
tf.keras.backend.set_epsilon(1)

In [ ]:
clf = KerasRegressor(
    model=get_clf_model,
    hidden_layer_sizes=(16,),
    optimizer="adam",
    optimizer__learning_rate=0.001,
    epochs=200,
    batch_size=16,
    validation_split=0.2,
    verbose=1,
)

In [ ]:
forecaster = make_reduction(clf, window_length=20, strategy="recursive")
forecaster.fit(y_train)
y_pred = forecaster.predict(fh=fh)

In [ ]:
forecaster

In [ ]:
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
plt.show()